In [1]:
from langchain_openai.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains.conversation.base import ConversationChain

In [2]:
chat = ChatOpenAI(model="gpt-3.5-turbo-0125")
memory = ConversationBufferMemory()
chain = ConversationChain(
    llm=chat,
    memory=memory,
    verbose=True
)

In [4]:
chain.predict(input="olá")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: olá
AI:

> Finished chain.


'Olá! Como posso ajudá-lo hoje?'

In [5]:
chain.run("olá")

d:\Udemy\2025\langchain\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: olá
AI: Olá! Como posso ajudá-lo hoje?
Human: olá
AI:

> Finished chain.


'Olá novamente! Como posso ajudá-lo hoje?'

In [6]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("""
Essa é uma conversa amigável entre um humano e uma IA

Conversa atual:
{history}
Human: {input}
AI:"""
)

In [7]:
chat = ChatOpenAI(model="gpt-3.5-turbo-0125")
memory = ConversationBufferMemory()
chain = ConversationChain(
    prompt=prompt_template,
    llm=chat,
    memory=memory,
    verbose=True
)

In [9]:
chain.predict(input="Oi")



> Entering new ConversationChain chain...
Prompt after formatting:

Essa é uma conversa amigável entre um humano e uma IA

Conversa atual:

Human: Oi
AI:

> Finished chain.


'Olá! Como posso ajudar você hoje?'

### LLMChain

In [10]:
from langchain_openai.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains.llm import LLMChain

chat = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [11]:
prompt = PromptTemplate.from_template(
    """
    Escolha o melhor nome para mim sobre uma empresa que desenvolve soluções em {produto}
    """
)

chain = LLMChain(llm=chat, prompt=prompt)
produto = "LLMs com LangChain"
chain.run(produto)

'"LangChain Solutions"'

### SimpleSequentialChain

In [12]:
from langchain.chains import SimpleSequentialChain

chat = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [14]:
prompt = PromptTemplate.from_template(
    """
    Escolha o melhor nome para mim sobre uma empresa que desenvolve soluções em {produto}
    """
)
chain_nome = LLMChain(llm=chat, prompt=prompt)

prompt_descricao = PromptTemplate.from_template(
    """
    Sobre a empresa com nome {nome_empresa}.
    Informe uma descrição de até 20 palavras.
    """
)

chain_descricao = LLMChain(llm=chat, prompt=prompt_descricao)


In [15]:
chain = SimpleSequentialChain(
    chains=[chain_nome, chain_descricao],
    verbose=True
)

chain.run(input=produto)



> Entering new SimpleSequentialChain chain...
LangChain Solutions
LangChain Solutions é uma empresa de tecnologia que oferece soluções de blockchain e desenvolvimento de software personalizado.

> Finished chain.


'LangChain Solutions é uma empresa de tecnologia que oferece soluções de blockchain e desenvolvimento de software personalizado.'

### SequentialChain

In [16]:
from langchain.chains import SequentialChain

chat = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [17]:
prompt = PromptTemplate.from_template(
    """
    Qual um bom nome de empresa que desenvolve soluções em {produto}?
    """
)
chain_nome = LLMChain(llm=chat, prompt=prompt, output_key="nome_empresa")

prompt_descricao = PromptTemplate.from_template(
    """
    Sobre a empresa com o nome {nome_empresa} e que 
    produz seguinte {produto}.
    Informe uma descrição de até 30 palavras de atividades dessa empresa
    """
)
chain_descricao = LLMChain(llm=chat, prompt=prompt_descricao, 
                           output_key="descricao_empresa")

prompt_traducao = PromptTemplate.from_template(
    """
    Crie um nome em espanhol para a empresa de nome {nome_empresa}
    que possui a seguinte descrição {descricao_empresa}
    """
)
chain_traducao = LLMChain(llm=chat, prompt=prompt_traducao, 
                          output_key="nome_espanhol")

In [18]:
chain = SequentialChain(
    chains=[chain_nome, chain_descricao, chain_traducao],
    input_variables=["produto"],
    output_variables=["nome_empresa", "descricao_empresa", "nome_espanhol"],
    verbose=True
)

resposta = chain.invoke({"produto": produto})



> Entering new SequentialChain chain...

> Finished chain.


In [19]:
resposta["produto"]

'LLMs com LangChain'

In [20]:
resposta["nome_empresa"]

'"LangSolutions"'

In [21]:
resposta["descricao_empresa"]

'A empresa LangSolutions produz LLMs com LangChain, oferecendo soluções inovadoras de linguagem natural para empresas em busca de otimização de processos e comunicação.'

In [22]:
resposta["nome_espanhol"]

'SolucionesLingüísticasLangChain'